# Evolver Loop 7 Analysis

## Key Questions:
1. Why is there a 17-point gap between Zaburo (87.99) and pre-optimized baselines (70.6)?
2. What techniques can close this gap?
3. What's the per-N breakdown of the gap?

In [ ]:
import pandas as pd
import numpy as np
import json
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union

getcontext().prec = 25
scale_factor = Decimal('1e15')

print('Analysis notebook initialized')

In [ ]:
# Load Zaburo solution (our best valid)
zaburo_path = '/home/code/experiments/005_zaburo_rowbased/submission.csv'
zaburo_df = pd.read_csv(zaburo_path)

# Load pre-optimized baseline (has overlaps but shows what's possible)
baseline_path = '/home/nonroot/snapshots/santa-2025/21329067673/submission/submission.csv'
baseline_df = pd.read_csv(baseline_path)

print(f'Zaburo: {len(zaburo_df)} rows')
print(f'Baseline: {len(baseline_df)} rows')

In [ ]:
def parse_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return float(val[1:])
    return float(val)

def compute_score_for_n(df, n):
    """Compute score for a specific N value."""
    group = df[df['id'].str.startswith(f'{n:03d}_')]
    if len(group) == 0:
        return None
    
    # Get all coordinates
    xs = [parse_value(x) for x in group['x']]
    ys = [parse_value(y) for y in group['y']]
    degs = [parse_value(d) for d in group['deg']]
    
    # Tree template
    TREE_TEMPLATE = [
        (0.0, 0.8), (0.125, 0.5), (0.0625, 0.5), (0.2, 0.25), (0.1, 0.25),
        (0.35, 0.0), (0.075, 0.0), (0.075, -0.2), (-0.075, -0.2), (-0.075, 0.0),
        (-0.35, 0.0), (-0.1, 0.25), (-0.2, 0.25), (-0.0625, 0.5), (-0.125, 0.5),
    ]
    
    all_points = []
    for x, y, deg in zip(xs, ys, degs):
        base = Polygon(TREE_TEMPLATE)
        rotated = affinity.rotate(base, deg, origin=(0, 0))
        translated = affinity.translate(rotated, xoff=x, yoff=y)
        coords = list(translated.exterior.coords)
        all_points.extend(coords)
    
    all_points = np.array(all_points)
    minx, miny = all_points.min(axis=0)
    maxx, maxy = all_points.max(axis=0)
    side = max(maxx - minx, maxy - miny)
    return (side ** 2) / n

print('Score function defined')

In [ ]:
# Compare per-N scores
print('Computing per-N scores...')
print('='*80)

zaburo_scores = {}
baseline_scores = {}
gaps = []

for n in range(1, 201):
    z_score = compute_score_for_n(zaburo_df, n)
    b_score = compute_score_for_n(baseline_df, n)
    
    zaburo_scores[n] = z_score
    baseline_scores[n] = b_score
    
    if z_score and b_score:
        gap = z_score - b_score
        gaps.append((n, gap, z_score, b_score))

print(f'Zaburo total: {sum(zaburo_scores.values()):.6f}')
print(f'Baseline total: {sum(baseline_scores.values()):.6f}')
print(f'Gap: {sum(zaburo_scores.values()) - sum(baseline_scores.values()):.6f}')

In [ ]:
# Sort by gap (largest first)
gaps_sorted = sorted(gaps, key=lambda x: x[1], reverse=True)

print('\nTop 20 N values with largest gaps (Zaburo - Baseline):')
print('='*80)
print(f'{"N":>4} | {"Gap":>10} | {"Zaburo":>12} | {"Baseline":>12}')
print('-'*80)
for n, gap, z, b in gaps_sorted[:20]:
    print(f'{n:4d} | {gap:10.6f} | {z:12.6f} | {b:12.6f}')

print(f'\nSum of top 20 gaps: {sum(g[1] for g in gaps_sorted[:20]):.6f}')

In [ ]:
# Analyze where the gap comes from
print('\nGap distribution by N range:')
print('='*80)

ranges = [(1, 10), (11, 50), (51, 100), (101, 150), (151, 200)]
for start, end in ranges:
    range_gaps = [g for g in gaps if start <= g[0] <= end]
    total_gap = sum(g[1] for g in range_gaps)
    avg_gap = total_gap / len(range_gaps) if range_gaps else 0
    print(f'N={start:3d}-{end:3d}: Total gap = {total_gap:8.4f}, Avg gap = {avg_gap:.6f}')

In [ ]:
# Key insight: The baseline has overlaps but shows what's POSSIBLE
# The gap is ~17 points, mostly from small N values

print('\n' + '='*80)
print('KEY INSIGHT:')
print('='*80)
print(f'''
The pre-optimized baseline scores 70.6 but has overlaps.
Zaburo's row-based approach scores 87.99 but is VALID.

The gap of ~17 points comes from:
1. Zaburo uses simple row-based placement (not optimized)
2. Baseline uses sophisticated optimization (but creates overlaps)

To close the gap, we need:
1. Start from Zaburo's VALID solution
2. Apply optimization that MAINTAINS validity
3. Use fractional translation (0.001 to 0.00001 step sizes)
4. Focus on small N values first (highest gap per tree)
''')

In [ ]:
# Check what the SA experiment improved
sa_metrics_path = '/home/code/experiments/006_sa_from_scratch/metrics.json'
with open(sa_metrics_path) as f:
    sa_metrics = json.load(f)

print('SA Experiment Results:')
print('='*80)
print(f"Initial score: {sa_metrics['initial_score']:.6f}")
print(f"Final score: {sa_metrics['cv_score']:.6f}")
print(f"Improvement: {sa_metrics['improvement']:.6f}")

# Per-N improvements from SA
sa_per_n = sa_metrics.get('per_n_scores', {})
if sa_per_n:
    print('\nSA improvements by N (where improved):')
    for n in range(1, 11):
        z_score = zaburo_scores.get(n)
        sa_score = sa_per_n.get(str(n))
        if z_score and sa_score:
            diff = z_score - sa_score
            if diff > 0.0001:
                print(f'  N={n}: {z_score:.6f} -> {sa_score:.6f} (improved by {diff:.6f})')